In [ ]:
#=================================================================================================================
# WISE utilities:
#=================================================================================================================

#-----------------------------------------------------------------------------------------------------------------
def scale_image(scale, *args):
    """
    Returns the scaling needed to download wise images.
    If scale == 'mpc', *args: img_z, img_pix, img_mpc
    If scale == 'arcsec', *args: img_z, img_pix, img_arcsec
    """
    
    arcsecs_in_rad = 360.*3600./(2.*np.pi)
        
    if scale == 'mpc':
        img_z, img_pix, img_mpc = args
        ad_dist = Planck13.angular_diameter_distance(img_z).value
        img_rads = img_mpc/ad_dist
        img_arcsec = img_rads*arcsecs_in_rad
    elif scale == 'arcsec':
        img_z, img_pix, img_arcsec = args
        img_rads = img_arcsec/arcsecs_in_rad
        ad_dist = Planck13.angular_diameter_distance(img_z).value
        img_mpc = img_rads*ad_dist
        
    if img_z == -1.:
        print('*** ERROR: wrong redshift equal to -1.')

    img_arcmin = img_arcsec/60.
    sdss_scale = img_arcsec/img_pix
        
    return sdss_scale, img_arcsec, img_arcmin, img_mpc


#-----------------------------------------------------------------------------------------------------------------
def download_wise_fits(img_id, img_ra, img_dec, img_arcsec, wise_band, verbose=False):
    """
    Routine to download WISE fits and store them in img_dir.
    
    http://wise2.ipac.caltech.edu/docs/release/allwise/expsup/index.html
    WISE (...) mapped the sky at 3.4, 4.6, 12, and 22 μm (W1, W2, W3, W4) in 2010
    with an angular resolution of 6.1", 6.4", 6.5", & 12.0" in the four bands.
    
    fits file info: ( hdu_list.info() )
    No.    Name         Type      Cards   Dimensions   Format
    0    PRIMARY     PrimaryHDU      67   (XXX, XXX)   float32 
    """
           
    fits_file = '%s/FITS/IMG_%06d_WISE,_%04d_arcsec,_band_%i.fits' % (img_dir, img_id, img_arcsec, wise_band)

    count = 0
    while (not os.path.isfile(fits_file)) and (count < 50):
        count += 1
        try:
            base_url = 'https://irsa.ipac.caltech.edu/ibe/search/wise/allwise/p3am_cdd?columns=coadd_id,'
            query_band = 'band&where=band%20IN%20' + '(%i)' % (wise_band)
            query_pos = '&POS=%.4f,%.4f&INTERSECT=CENTER&mcen' % (img_ra, img_dec)
            final_url = base_url + query_band + query_pos
            if verbose: 
                print('Query URL:\n', final_url)
            urlretrieve(final_url, '%s/query_url.dat' % tmp_dir)
            if verbose: 
                print('\nData received:')
                !cat {tmp_dir}/query_url.dat
            !tail -n 1 {tmp_dir}/query_url.dat > {tmp_dir}/query_data.dat
            meta_data = np.genfromtxt('%s/query_data.dat' % tmp_dir, dtype='str')

            params = {'coadd_id':meta_data[-3] , 'band':wise_band}
            params['coaddgrp'] = params['coadd_id'][:2]
            params['coadd_ra'] = params['coadd_id'][:4]

            path = str.format('{coaddgrp:s}/{coadd_ra:s}/{coadd_id:s}/{coadd_id:s}-w{band:1d}-int-3.fits?',**params)
            cutoff_url = 'center=%.4f,%.4fdeg&size=%i,%iarcsec&gzip=false' % (img_ra, img_dec, img_arcsec, img_arcsec)

            img_url = 'https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/' + path + cutoff_url

            if verbose:
                print('\nDownloading fits image from:\n%s' % img_url)
            urlretrieve(img_url, fits_file)
            try:
                check_fits = fits.getdata(fits_file)
                if verbose:
                    print('\nFits image saved in:\n%s\n\n******************************\n' % fits_file)
            except:
                os.remove(fits_file)
        except:
            pass
            

#-----------------------------------------------------------------------------------------------------------------
def get_wise_image_data(img_id, img_arcsec, wise_band):
    """
    Returns the numpy array matrix obtained from the fits file.
    """

    fits_file = '%s/FITS/IMG_%06d_WISE,_%04d_arcsec,_band_%i.fits' % (img_dir, img_id, img_arcsec, wise_band)
    try:
        img_data = fits.getdata(fits_file)
    except:
        if os.path.exists(fits_file):
            os.remove(fits_file)
        return np.zeros((50, 10)) # triggers bad_ratio
    img_data = np.rot90(np.rot90(np.fliplr(img_data)))
    #img_data = np.array(img_data, dtype=np.float)
        
    return img_data


#-----------------------------------------------------------------------------------------------------------------
def wise_fits_to_png(img_id, img_arcsec, wise_band):
    """
    Produces a raw image in img_dir from the fits file.
    """
               
    png_file  = '%s/PNG/IMG_%06d_WISE,_%04d_arcsec,_band_%i.png' % (img_dir, img_id, img_arcsec, wise_band)
    img_data = get_wise_image_data(img_id, img_arcsec, wise_band)
    fig = plot_image_data(img_data)
    fig.savefig(png_file)
    